In [1]:
import warnings
warnings.filterwarnings("ignore")
from shapely import wkt
import glob


%load_ext autoreload
%autoreload 2
%env PROJ_LIB=/home/wwymak/anaconda3/envs/solaris/share/proj
%matplotlib inline

import skimage

from tqdm import tqdm
import numpy as np
import cv2
from functools import partial

from fastai.imports import *
from fastai.vision import *
from fastai.metrics import dice
from fastai.callbacks import *

from joblib import Parallel, delayed
import torch.nn.functional as F
import torch
import functools, traceback
def gpu_mem_restore(func):
    """Reclaim GPU RAM if CUDA out of memory happened, or execution was interrupted"""
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except:
            type, val, tb = sys.exc_info()
            traceback.clear_frames(tb)
            raise type(val).with_traceback(tb) from None
    return wrapper

os.environ['FASTAI_TB_CLEAR_FRAMES']="1"

from catalyst.contrib.criterion import FocalLossMultiClass, DiceLoss, BCEDiceLoss, LovaszLossBinary,LovaszLossMultiClass

from importlib import reload

env: PROJ_LIB=/home/wwymak/anaconda3/envs/solaris/share/proj


In [2]:
data_dir = Path("/media/wwymak/Storage/xView2")
mask_dir = data_dir /"mask_full_size"

train_images_crops = data_dir/"train_crops"
# train_mask_crops = data_dir/"mask_crops"
train_mask_crops = data_dir/"mask_crops_single_channel"
test_images_crops = data_dir / "test_crops"
test_mask_crops = data_dir / "test_mask_crops_single_channel"
test_masks_edt = data_dir / "test_masks_edt"
classifcation_crop_dir = data_dir/"classification_crops_post"

data_dir = Path("/media/wwymak/Storage/xView2")
mask_dir = data_dir /"mask_full_size"
models_path = data_dir / "models"
train_images_crops = data_dir/"train_crops"
mask_dir_edt = data_dir /"mask_full_size_edt"
train_mask_crops = data_dir/"mask_crops_single_channel"
label_dir = data_dir/"train"/"labels"
mask_crops_edt = data_dir/"mask_crops_edt"
test_crops = data_dir/"test_crops"

In [20]:
labels = pd.read_csv(data_dir/"test_polygons_edt.csv")

labels["crop_filename"] = labels.polygon_id.apply(lambda x: f"{x}.png")
labels.head()

,geometry,value,polygon_id,img_id,crop_filename
0,"POLYGON ((612 10, 612 12, 608 12, 608 16, 606 ...",255.0,b44cd049-e8f6-4af9-84d6-c1327af8b46d,test_localization_00000_prediction.png,b44cd049-e8f6-4af9-84d6-c1327af8b46d.png
1,"POLYGON ((736 0, 736 2, 738 2, 738 4, 742 4, 7...",255.0,0fa46888-a175-4802-ad94-93425524dc71,test_localization_00000_prediction.png,0fa46888-a175-4802-ad94-93425524dc71.png
2,"POLYGON ((572 44, 572 48, 570 48, 570 50, 568 ...",255.0,9807c798-dcc7-4afc-859d-c55c5d5873ef,test_localization_00000_prediction.png,9807c798-dcc7-4afc-859d-c55c5d5873ef.png
3,"POLYGON ((556 76, 556 78, 552 78, 552 82, 550 ...",255.0,ceb99454-2f1c-48a5-ae5f-d5e64020711e,test_localization_00000_prediction.png,ceb99454-2f1c-48a5-ae5f-d5e64020711e.png
4,"POLYGON ((734 38, 734 40, 732 40, 732 42, 728 ...",255.0,298c5b42-88d1-4b86-bbba-822939099be8,test_localization_00000_prediction.png,298c5b42-88d1-4b86-bbba-822939099be8.png


In [13]:
class CombinedLoss(nn.Module):

    def __init__(self, gamma=2., alpha=0.25):
        super(CombinedLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha

    def forward(self, input, target, **kwargs):
        y = to_onehot(target, input.size(-1))
        logit = F.softmax(input)
                     
        focal_loss = FocalLossMultiClass(gamma=self.gamma, alpha=self.alpha)
        
        total_loss = focal_loss(logit, target) #+ 0.2 * dice_loss(logit, y)
        return total_loss

In [14]:
bs=32
size=64
src = (ImageList
       .from_df(classification_labels, path=classifcation_crop_dir, cols=['crop_filename'])
       # this doesn't really matter, since we're just using the suff for infernce
       .split_by_rand_pct(0.2).label_from_df(cols='label'))
data = (src
    .transform(get_transforms(do_flip=True, 
         flip_vert=True, 
         max_rotate=180, 
         max_zoom=1.2, 
         max_lighting=0.5,
         max_warp=0.2, 
         p_affine=0.75, 
         p_lighting=0.75), size=size, tfm_y=False)
    .add_test_folder(damage_crops_test_folder)
    .databunch(bs=bs)
    .normalize(imagenet_stats))

learn = cnn_learner(data,models.resnet50)
learn.model_dir = models_path
learn.loss_func =CombinedLoss(alpha=0.75, gamma=4);
learn.to_fp16();
learn.load('29nov_focal_alpha075_gamma_4')

Learner(data=ImageDataBunch;

Train: LabelList (131073 items)
x: ImageList
Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64)
y: CategoryList
0,0,0,0,0
Path: /media/wwymak/Storage/xView2/classification_crops_post;

Valid: LabelList (32768 items)
x: ImageList
Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64)
y: CategoryList
0,0,0,0,0
Path: /media/wwymak/Storage/xView2/classification_crops_post;

Test: LabelList (33614 items)
x: ImageList
Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64)
y: EmptyLabelList
,,,,
Path: /media/wwymak/Storage/xView2/classification_crops_post, model=Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1

In [17]:
def to_onehot(indices, num_classes):
    """Convert a tensor of indices of any shape `(N, ...)` to a
    tensor of one-hot indicators of shape `(N, num_classes, ...) and of type uint8. Output's device is equal to the
    input's device`.
    """
    onehot = torch.zeros(indices.shape[0], num_classes, *indices.shape[1:],
#                              dtype=torch.uint8,
                         device=indices.device)
    return onehot.scatter_(1, indices.unsqueeze(1), 1)
class CombinedLoss(nn.Module):

    def __init__(self, gamma=2., alpha=0.25):
        super(CombinedLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha

    def forward(self, input, target, **kwargs):
        y = to_onehot(target, input.size(-1))
        logit = F.softmax(input)
                     
        focal_loss = FocalLossMultiClass(gamma=self.gamma, alpha=self.alpha)
#         bce_dice = BCEDiceLoss()
#         lovaz_loss = LovaszLossBinary()
#         dice_loss = DiceLoss(activation='Softmax')
        
        total_loss = focal_loss(logit, target) #+ 0.2 * dice_loss(logit, y)
        return total_loss

In [18]:
predictions, y, losses = learn.get_preds(ds_type=DatasetType.Test, with_loss=True)
y = torch.argmax(predictions, dim=1)

In [19]:


polygon_ids = [x.name.replace('.png', '') for x in data.test_dl.dataset.items]
prediction_cls = [x+1 for x in y.numpy()]

test_results = pd.DataFrame(data={'polygon_id': polygon_ids, 'damage_cls': prediction_cls})
test_results = labels.merge(test_results, left_on='polygon_id', right_on='polygon_id')
test_results.geometry = test_results.geometry.apply(lambda x: wkt.loads(x).exterior)
test_results.img_id = test_results.img_id.str.replace('localization', 'damage')

test_results.head()

,geometry,value,polygon_id,img_id,crop_filename,damage_cls
0,"LINEARRING (612 10, 612 12, 608 12, 608 16, 60...",255.0,b44cd049-e8f6-4af9-84d6-c1327af8b46d,test_damage_00000_prediction.png,b44cd049-e8f6-4af9-84d6-c1327af8b46d.png,1
1,"LINEARRING (736 0, 736 2, 738 2, 738 4, 742 4,...",255.0,0fa46888-a175-4802-ad94-93425524dc71,test_damage_00000_prediction.png,0fa46888-a175-4802-ad94-93425524dc71.png,1
2,"LINEARRING (572 44, 572 48, 570 48, 570 50, 56...",255.0,9807c798-dcc7-4afc-859d-c55c5d5873ef,test_damage_00000_prediction.png,9807c798-dcc7-4afc-859d-c55c5d5873ef.png,1
3,"LINEARRING (556 76, 556 78, 552 78, 552 82, 55...",255.0,ceb99454-2f1c-48a5-ae5f-d5e64020711e,test_damage_00000_prediction.png,ceb99454-2f1c-48a5-ae5f-d5e64020711e.png,1
4,"LINEARRING (734 38, 734 40, 732 40, 732 42, 72...",255.0,298c5b42-88d1-4b86-bbba-822939099be8,test_damage_00000_prediction.png,298c5b42-88d1-4b86-bbba-822939099be8.png,1


In [36]:
test_results.damage_cls.value_counts(normalize=True)

1    0.793657
4    0.078063
3    0.070417
2    0.057863
Name: damage_cls, dtype: float64

In [21]:
from cv2 import fillPoly, imwrite


def create_image(img_id, output_path):
    mask_img = np.zeros((1024, 1024, 1), np.uint8)
    img_polys = test_results[test_results.img_id == img_id]
    if len(img_polys) > 0:
        for r in img_polys.iterrows():
            row = r[1]
            poly_np = np.array(row.geometry.coords, np.int32)
            fillPoly(mask_img, [poly_np], row['damage_cls'])

    imwrite(str(output_path / img_id), mask_img)
    return mask_img

In [29]:

output_path = data_dir/"test_masks_edt"

img_ids = [re.findall(r'\d+', x.name )[0] for x in (data_dir/"test"/"images").ls() if 'post' in x.name]
img_ids = [f"test_damage_{x}_prediction.png" for x in img_ids]

outputs = [create_image(img_id, output_path) for img_id in tqdm(img_ids)]

100%|██████████| 933/933 [00:06<00:00, 136.34it/s]


In [30]:
from submission_utils import load_and_validate_image
_=[load_and_validate_image(x) for x in output_path.ls()]

In [32]:
seg_results = labels.copy()
seg_results["damage_cls"] = 1
seg_results.geometry = seg_results.geometry.apply(lambda x: wkt.loads(x).exterior)
seg_results.head()

,geometry,value,polygon_id,img_id,crop_filename,damage_cls
0,"LINEARRING (612 10, 612 12, 608 12, 608 16, 60...",255.0,b44cd049-e8f6-4af9-84d6-c1327af8b46d,test_localization_00000_prediction.png,b44cd049-e8f6-4af9-84d6-c1327af8b46d.png,1
1,"LINEARRING (736 0, 736 2, 738 2, 738 4, 742 4,...",255.0,0fa46888-a175-4802-ad94-93425524dc71,test_localization_00000_prediction.png,0fa46888-a175-4802-ad94-93425524dc71.png,1
2,"LINEARRING (572 44, 572 48, 570 48, 570 50, 56...",255.0,9807c798-dcc7-4afc-859d-c55c5d5873ef,test_localization_00000_prediction.png,9807c798-dcc7-4afc-859d-c55c5d5873ef.png,1
3,"LINEARRING (556 76, 556 78, 552 78, 552 82, 55...",255.0,ceb99454-2f1c-48a5-ae5f-d5e64020711e,test_localization_00000_prediction.png,ceb99454-2f1c-48a5-ae5f-d5e64020711e.png,1
4,"LINEARRING (734 38, 734 40, 732 40, 732 42, 72...",255.0,298c5b42-88d1-4b86-bbba-822939099be8,test_localization_00000_prediction.png,298c5b42-88d1-4b86-bbba-822939099be8.png,1


In [34]:
def create_image(img_id, output_path, test_results_dataframe):
    mask_img = np.zeros((1024, 1024, 1), np.uint8)
    img_polys = test_results_dataframe[test_results_dataframe.img_id == img_id]
    if len(img_polys) > 0:
        for r in img_polys.iterrows():
            row = r[1]
            poly_np = np.array(row.geometry.coords, np.int32)
            fillPoly(mask_img, [poly_np], row['damage_cls'])

    imwrite(str(output_path / img_id), mask_img)
    return mask_img

In [35]:
img_ids = [re.findall(r'\d+', x.name )[0] for x in (data_dir/"test"/"images").ls() if 'pre' in x.name]
img_ids = [f"test_localization_{x}_prediction.png" for x in img_ids]

outputs = [create_image(img_id, output_path, seg_results) for img_id in tqdm(img_ids)]
_=[load_and_validate_image(x) for x in output_path.ls()]


100%|██████████| 933/933 [00:06<00:00, 134.59it/s]


In [37]:
len(set([x.name for x in output_path.ls()]))

1866

In [38]:
output_path

PosixPath('/media/wwymak/Storage/xView2/test_masks_edt')